# Make TREK-150 Our Data

In [163]:
import csv, os, sys, re, string, json, glob, shutil, random, math, pprint, ast, pickle
from tqdm import tqdm
from ast import literal_eval

import cv2
import pandas as pd
import numpy as np

from IPython.display import HTML as html_print
from IPython.display import Markdown
from IPython.display import clear_output

# from IPython.display import Image
from PIL import Image, ImageDraw, ImageFont
from ipywidgets import Video

import torch
from transformers import AutoTokenizer
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
sns.set()
from matplotlib import pyplot as plt
import matplotlib.patches as patches
%pylab inline

import re
from IPython.display import Image as ipy_Image
from datetime import datetime
from datetime import timedelta

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Use our own built-in customized COCO API.
sys.path.insert(0, "../cocoapi")

In [3]:
# from pycocotools.coco import COCO
# from pycocotools.cocoeval import COCOeval
from PythonAPI.pycocotools.coco import COCO
from PythonAPI.pycocotools.cocoeval import COCOeval

import inspect
print("COCO API at:     {}".format(inspect.getfile(COCO)))
print("COCOeval API at: {}".format(inspect.getfile(COCOeval)))

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%reload_ext autoreload
%autoreload 2

COCO API at:     /home/telinwu/research/project_jarvis/all_visualizations/../cocoapi/PythonAPI/pycocotools/coco.py
COCOeval API at: /home/telinwu/research/project_jarvis/all_visualizations/../cocoapi/PythonAPI/pycocotools/cocoeval.py


In [4]:
def check_gt_pred_match_on_file(
    coco_gt_file,
    coco_pred_file,
):
    print("Checking GT and Pred files matching...")
    if type(coco_gt_file) is str:
        coco_gt_data = json.load(open(coco_gt_file))
    else:
        coco_gt_data = coco_gt_file
    if type(coco_pred_file) is str:
        coco_pred_data = json.load(open(coco_pred_file))
    elif type(coco_pred_file) == np.ndarray:
        _coco = COCO(verbose=False)
        coco_pred_data = _coco.loadNumpyAnnotations(coco_pred_file)
    else:
        coco_pred_data = coco_pred_file
    gt_image_ids = {x["id"]: True for x in coco_gt_data["images"]}
    pred_image_ids = [x["image_id"] for x in coco_pred_data]
    pred_image_ids = list(set(pred_image_ids))
    for _id in pred_image_ids:
        if _id not in gt_image_ids:
            raise ValueError(
                "\nGT file: {}\nPred file: {}\nThey do not match!?".format(coco_gt_file, coco_pred_file))
        pass
    print("Checking complete!")
    pass  # Passed.


def naive_ego4d_scod_coco_results(
    coco_gt_file,
    coco_pred_file,
    original_coco_gt_file=None,
    top_k=None,
    ignore_summaries=None,
    verbose=False,
):  
    if original_coco_gt_file is not None:
        coco_gt = COCO(annotation_file=original_coco_gt_file, verbose=verbose)
        coco_gt = coco_gt.loadRes(coco_gt_file)
        check_gt_pred_match_on_file(original_coco_gt_file, coco_pred_file)
    else:
        coco_gt = COCO(annotation_file=coco_gt_file, verbose=verbose)
        check_gt_pred_match_on_file(coco_gt_file, coco_pred_file)
    coco_dt = coco_gt.loadRes(coco_pred_file)
    coco_eval = COCOeval(coco_gt, coco_dt, "bbox")
    coco_eval.params.verbose = verbose
    if top_k is not None:
        coco_eval.params.maxDets = [top_k, top_k, top_k]
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize(ignore_summaries=ignore_summaries)
    return coco_eval

In [142]:
def process_one_trek150_video(
    video_id,
    trek150_data_root="/local1/bryanzhou008/jarvis/epic_kitchen/clean/fpv-tracking-baselines/LTMU-H/TREK-150",
    frame_num_step=10,
    image_id_accum=0,
    annot_id_accum=0,
    coco_format_images=[],
    coco_format_annots=[],
    narrations_info_dict=None,
    token_positive_method=None,
    action_class_mappings=None,
    per_video_start_frames=None,
    verbose=False,
):
    video_dir = os.path.join(trek150_data_root, video_id)
    assert os.path.exists(video_dir)
    frames_file = os.path.join(video_dir, "frames.txt")
    assert os.path.exists(frames_file)
    frame_nums = open(frames_file).readlines()
    frame_nums = [int(x.strip()) for x in frame_nums]
    gt_bboxes_file = os.path.join(video_dir, "groundtruth_rect.txt")
    gt_bboxes = open(gt_bboxes_file).readlines()
    gt_bboxes = [[float(b) for b in x.strip().split(",")] for x in gt_bboxes]

    assert len(gt_bboxes) == len(frame_nums)
    
    if narrations_info_dict is not None:
        start_frame = frame_nums[0]
        stop_frame = frame_nums[-1]
        mid_frame = (start_frame + stop_frame) // 2
        frame_narr = []
        for narrations_info in narrations_info_dict:
            start_fr, stop_fr, mid_fr, narr = narrations_info
            diff_fr = abs(mid_frame - mid_fr)
            overlapping = len(range( max(start_frame, start_fr), min(stop_frame, stop_fr)+1 ))
            overlapping = overlapping / (stop_frame - start_frame + 1)
            # frame_narr.append((diff_fr, start_fr, stop_fr, narr))
            # First compute the overlapping, then the stop frame `prior` range.
            frame_narr.append((-overlapping, abs(stop_fr-start_frame), start_fr, stop_fr, narr))
        frame_narr = sorted(frame_narr)[:5]
        frame_narr = [(x, y, z) for _, _, x, y, z in frame_narr]
        caption = frame_narr[0][-1]
        if verbose:
            print(start_frame, stop_frame)
            pprint.pprint(frame_narr)
            print(caption)
            raise
        
        action_target_file = os.path.join(video_dir, "action_target.txt")
        assert os.path.exists(action_target_file)
        action_targets = open(action_target_file).readlines()
        action_targets = [int(x.strip()) for x in action_targets]
        verb = action_targets[0]
        act_noun, target_noun = action_targets[1], action_targets[2]
        
        if token_positive_method == "gt":
            target_nouns_allowed = action_class_mappings["noun"][target_noun]
            tokens = caption.split(" ")
            begin_pos, end_pos = 0, 0
            target_poss = []
            gt_word_found = False
            for token in tokens:
                end_pos = begin_pos + len(token)
                for target_noun_allowed in target_nouns_allowed:
                    if (
                        token == target_noun_allowed and
                        [begin_pos, end_pos] not in target_poss
                    ):
                        target_poss.append([begin_pos, end_pos])
                    elif "-" in target_noun_allowed:
                        if (
                            token == target_noun_allowed.split("-")[0] or
                            token == target_noun_allowed.split("-")[1] and
                            [begin_pos, end_pos] not in target_poss
                        ):
                            target_poss.append([begin_pos, end_pos])
                    if target_noun_allowed in token:
                        gt_word_found = True
                begin_pos = end_pos + 1
                pass
            target_poss = sorted(target_poss)
            tokens_positive = target_poss
            if not gt_word_found:
                print(caption)
                print(target_nouns_allowed)
                print(action_targets)
                print(start_frame, stop_frame)
                pprint.pprint(frame_narr)
                print(video_id)
                caption = action_class_mappings["verb"][verb][0] \
                    + " " + action_class_mappings["noun"][target_noun][0]
                print(caption)
                begin_pos = len(action_class_mappings["verb"][verb][0]) + 1
                end_pos = begin_pos + len(action_class_mappings["noun"][target_noun][0])
                target_poss.append([begin_pos, end_pos])
                print(target_poss)
                print("-"*50)
                # raise
        # raise

    frame_num_idx = 0
    while frame_num_idx < len(frame_nums):
        # Per video start frame.
        if (
            per_video_start_frames is not None
            and per_video_start_frames[video_id] is not None
            and frame_nums[frame_num_idx] < per_video_start_frames[video_id]
        ):
            frame_num_idx += 1
            continue
        if (
            per_video_start_frames is not None
            and frame_nums[frame_num_idx] == per_video_start_frames[video_id]
        ):
            print("Saved {} frames out of {} frames.".format(frame_num_idx, len(frame_nums)))
        
        if frame_num_idx >= len(frame_nums):
            break
        gt_bbox = gt_bboxes[frame_num_idx]
        while sum(gt_bbox) == -4:  # [-1, -1, -1, -1] no bbox case!
            if frame_num_idx >= len(frame_nums):
                break
            frame_num_idx += 1 # Just skip these frames!
            gt_bbox = gt_bboxes[frame_num_idx]
        if frame_num_idx >= len(frame_nums):
            break
        
        bbx, bby, bbh, bbw = gt_bbox
        frame_num = frame_nums[frame_num_idx]
        img_path = os.path.join(video_dir, "img", "frame_{:010d}.jpg".format(frame_num))
        file_name = os.path.join(video_id, "img", "frame_{:010d}.jpg".format(frame_num))
        assert os.path.exists(img_path)
        frame_image = Image.open(img_path)
        width, height = frame_image.size
        
        if narrations_info_dict is None:
            caption = "object_of_change"
            tokens_positive_eval = [[[0, 16]]]
            tokens_positive = [[0, 16]]
        else:
            if len(tokens_positive) == 0:
                tokens_positive = [[0, len(caption)]]
            tokens_positive_eval = [tokens_positive]
        
        image_dict = {
            "file_name": file_name,
            "height": height,
            "width": width,
            "id": image_id_accum,
            "caption": caption,
            "tokens_positive_eval": tokens_positive_eval,
        }
        coco_format_images.append(image_dict)
        annot_dict = {
            "segmentation": [],
            "area": bbh * bbw,
            "iscrowd": 0,
            "ignore": 0,
            "image_id": image_id_accum,
            "bbox": gt_bbox,
            "category_id": 1,
            "id": annot_id_accum,
            "tokens_positive": tokens_positive,
        }
        coco_format_annots.append(annot_dict)
        
        annot_id_accum += 1
        image_id_accum += 1
        frame_num_idx += frame_num_step

    return image_id_accum, annot_id_accum


def tracking_to_od_results_on_one_video(
    video_id,
    trek150_data_root="/local1/bryanzhou008/jarvis/epic_kitchen/clean/fpv-tracking-baselines/LTMU-H/TREK-150",
    pred_root="/local1/bryanzhou008/jarvis/epic_kitchen/clean/fpv-tracking-baselines/LTMU-H/TREK-150/LTMU-H/ope",
    frame_num_step=10,
    image_id_accum=0,
    coco_format_preds=[],
):
    video_dir = os.path.join(trek150_data_root, video_id)
    assert os.path.exists(video_dir)
    frames_file = os.path.join(video_dir, "frames.txt")
    assert os.path.exists(frames_file)
    frame_nums = open(frames_file).readlines()
    frame_nums = [int(x.strip()) for x in frame_nums]
    gt_bboxes_file = os.path.join(video_dir, "groundtruth_rect.txt")
    gt_bboxes = open(gt_bboxes_file).readlines()
    gt_bboxes = [[float(b) for b in x.strip().split(",")] for x in gt_bboxes]
    preds_file = os.path.join(pred_root, "{}.txt".format(video_id))
    assert os.path.exists(preds_file)
    pred_bboxes = open(preds_file).readlines()
    pred_bboxes = [[float(b) for b in x.strip().split(",")] for x in pred_bboxes]

    assert len(gt_bboxes) == len(frame_nums) == len(pred_bboxes)

    frame_num_idx = 0
    while frame_num_idx < len(frame_nums):
        if frame_num_idx >= len(frame_nums):
            break
        gt_bbox = gt_bboxes[frame_num_idx]
        while sum(gt_bbox) == -4:  # [-1, -1, -1, -1] no bbox case!
            if frame_num_idx >= len(frame_nums):
                break
            frame_num_idx += 1 # Just skip these frames!
            gt_bbox = gt_bboxes[frame_num_idx]
        if frame_num_idx >= len(frame_nums):
            break

        pred_dict = {
            "image_id": image_id_accum,
            "category_id": 1,
            "bbox": pred_bboxes[frame_num_idx],
            "score": 1.0,
        }
        coco_format_preds.append(pred_dict)

        image_id_accum += 1
        frame_num_idx += frame_num_step

    return image_id_accum

In [6]:
trek150_data_root = "/local1/bryanzhou008/jarvis/epic_kitchen/clean/fpv-tracking-baselines/LTMU-H/TREK-150"
video_id_file = os.path.join(os.path.join(trek150_data_root, "sequences.txt"))

video_ids = []
for line in open(video_id_file):
    video_ids.append(line.strip())

In [7]:
# Inits.
image_id_accum = 0
annot_id_accum = 0
frame_num_step = 10
coco_format_images = []
coco_format_annots = []

for video_id in tqdm(video_ids, desc="Processing TREK-150 Videos..."):
    image_id_accum, annot_id_accum = process_one_trek150_video(
        video_id=video_id,
        trek150_data_root="/local1/bryanzhou008/jarvis/epic_kitchen/clean/fpv-tracking-baselines/LTMU-H/TREK-150",
        frame_num_step=frame_num_step,
        image_id_accum=image_id_accum,
        annot_id_accum=annot_id_accum,
        coco_format_images=coco_format_images,
        coco_format_annots=coco_format_annots,
        narrations_info_dict=None,
        token_positive_method=None,
        action_class_mappings=None,
        per_video_start_frames=None,
        verbose=False,
    )
    
print(len(coco_format_images), len(coco_format_annots))

Processing TREK-150 Videos...: 100%|█████████████████████████████████████████████████████████████████████████████████| 150/150 [00:03<00:00, 40.96it/s]

9589 9589


In [8]:
def save_coco_file(
    coco_format_images,
    coco_format_annots,
    output_root="/local1/telinwu/research/resources/TREK-150/coco_annotations",
    output_name=None,
):
    output_coco_data = {
        "info": {
            "description": "TREK-150 to COCO formatted dataset",
            "version": "1.0.0",
            "year": 2023,
            "contributor": "Te-Lin Wu",
            "date_created": str(datetime.now()),
            "frame_num_step": 10,
            "trek150_data_root": trek150_data_root,
        },
        "licenses": [
            {
                "id": 1,
                "name": "Attribution-NonCommercial-ShareAlike License",
                "url": "http://creativecommons.org/licenses/by-nc-sa/2.0/"
            }
        ],
        "categories": [
            {
                "id": 1,
                "name": "object_of_change",
                "supercategory": "object"
            }
        ],
        "images": coco_format_images,
        "annotations": coco_format_annots,
    }

    output_to = (
        "{}"
        "/{}.json".format(output_root, output_name)
    )

    json.dump(
        output_coco_data,
        open(output_to, "w"),
        indent=4,
    )
    
    print("Saving coco file to: {}".format(output_to))

In [9]:
save_coco_file(
    coco_format_images,
    coco_format_annots,
    output_root="/local1/telinwu/research/resources/TREK-150/coco_annotations",
    output_name="trek150_stepwise_10_frames_od_only",
)

Saving coco file to: /local1/telinwu/research/resources/TREK-150/coco_annotations/trek150_stepwise_10_frames_od_only.json


In [10]:
# Inits.
image_id_accum = 0
frame_num_step = 10
coco_format_preds = []

for video_id in tqdm(video_ids, desc="Processing TREK-150 Videos..."):
    image_id_accum = tracking_to_od_results_on_one_video(
        video_id,
        trek150_data_root="/local1/bryanzhou008/jarvis/epic_kitchen/clean/fpv-tracking-baselines/LTMU-H/TREK-150",
        pred_root="/local1/bryanzhou008/jarvis/epic_kitchen/clean/fpv-tracking-baselines/LTMU-H/TREK-150/LTMU-H/ope",
        frame_num_step=frame_num_step,
        image_id_accum=image_id_accum,
        coco_format_preds=coco_format_preds,
    )
    
print(len(coco_format_preds))
outfile = "/local1/telinwu/research/resources/TREK-150/baselines/LTMU-H/ope/trek150_stepwise_10.json"
json.dump(
    coco_format_preds,
    open(outfile, "w"),
)

Processing TREK-150 Videos...: 100%|████████████████████████████████████████████████████████████████████████████████| 150/150 [00:00<00:00, 330.17it/s]

9589


# Some Results

In [208]:
coco_gt_file = (
    "/local1/telinwu/research/resources/TREK-150/coco_annotations"
    "/trek150_all_frames_with_first_mdnet_fail_narrated_gt.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/TREK-150/baselines"
    "/ego4d_scod_all_frames_narrated_gpt_v1_srl_arg1_with_tool_drop_null_ooc_tool_symb_conds_mask_no_comma"
    "/eval_ego4d_scod_val_finetuned_45K_on_TREK150_all_frames_new"
    "/eval/model_0045000/inference/narrated_ego4d_test/bbox.json"
)

ignore_summaries = {
    "area": ["small", "medium", "large"],
}
coco_eval = naive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    top_k=1,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

Checking GT and Pred files matching...
Checking complete!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=  1 ] = 0.564
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=  1 ] = 0.409
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.495
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.495
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.495


In [109]:
coco_gt_file = (
    "/local1/telinwu/research/resources/TREK-150/coco_annotations/trek150_stepwise_10_frames_od_only.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/TREK-150/baselines"
    "/LTMU-H/ope/trek150_stepwise_10.json"
)

ignore_summaries = {
    "area": ["small", "medium", "large"],
}
coco_eval = naive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    top_k=1,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=2.65s).
Accumulating evaluation results...
DONE (t=0.41s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=  1 ] = 0.364
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=  1 ] = 0.145
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.364
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.364
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.364


In [105]:
coco_gt_file = (
    "/local1/telinwu/research/resources/TREK-150/coco_annotations/trek150_stepwise_10_frames_od_only.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/TREK-150/baselines"
    "/naive_scod_finetuning_v1/eval_ego4d_scod_val_finetuned_45K"
    "/eval/model_0045000/inference/test/bbox.json"
)

ignore_summaries = {
    "area": ["small", "medium", "large"],
}
coco_eval = naive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    top_k=1,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=2.77s).
Accumulating evaluation results...
DONE (t=0.40s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=  1 ] = 0.194
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=  1 ] = 0.113
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.236
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.236
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.236


In [107]:
coco_gt_file = (
    "/local1/telinwu/research/resources/TREK-150/coco_annotations/trek150_stepwise_10_frames_od_only.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/TREK-150/baselines"
    "/naive_scod_finetuning_large_v1/eval_ego4d_scod_val_finetuned_45K"
    "/eval/model_0045000/inference/test/bbox.json"
)

ignore_summaries = {
    "area": ["small", "medium", "large"],
}
coco_eval = naive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    top_k=1,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=2.06s).
Accumulating evaluation results...
DONE (t=0.39s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=  1 ] = 0.286
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=  1 ] = 0.181
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.324
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.324
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.324


In [110]:
coco_gt_file = (
    "/local1/telinwu/research/resources/TREK-150/coco_annotations/trek150_stepwise_10_frames_od_only.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/TREK-150/baselines"
    "/narrated_scod_finetuning_large_v2/eval_ego4d_scod_val_finetuned_45K/"
    "/eval/model_0045000/inference/narrated_ego4d_test/bbox.json"
)

ignore_summaries = {
    "area": ["small", "medium", "large"],
}
coco_eval = naive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    top_k=1,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=2.38s).
Accumulating evaluation results...
DONE (t=0.40s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=  1 ] = 0.510
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=  1 ] = 0.372
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.471
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.471
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.471


In [117]:
coco_gt_file = (
    "/local1/telinwu/research/resources/TREK-150/coco_annotations/trek150_1st_frame_only_narrated_gt.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/TREK-150/baselines"
    "/narrated_scod_finetuning_large_v2/eval_ego4d_scod_val_finetuned_45K_on_TREK150_1st_frame_only/"
    "/eval/model_0045000/inference/narrated_ego4d_test/bbox.json"
)

ignore_summaries = {
    "area": ["small", "medium", "large"],
}
coco_eval = naive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    top_k=1,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=0.03s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=  1 ] = 0.412
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=  1 ] = 0.339
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.451
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.451
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.451


In [231]:
coco_gt_file = (
    "/local1/telinwu/research/resources/TREK-150/coco_annotations/trek150_stepwise_10_frames_od_only.json"
)
coco_pred_file = (
    "/local1/bryanzhou008/jarvis/project_jarvis/baselines/ego4d-eccv2022-solutions/scod/TREK-150-test-1.json"
)

ignore_summaries = {
    "area": ["small", "medium", "large"],
}
coco_eval = naive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    top_k=None,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

Checking GT and Pred files matching...
Checking complete!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.3791
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.5354
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.4012
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.4487
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.7180
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.7783


In [284]:
coco_gt_file = (
    "/local1/telinwu/research/resources/TREK-150/paper_data"
    "/narrated/gt_srl_arg1/trek150_all_frames_with_first_mdnet_fail_narrated_gt.json"
)
coco_pred_file = (
    "/local1/bryanzhou008/jarvis/project_jarvis/baselines/ego4d-eccv2022-solutions/scod/TREK-150-test-all_frame.json"
)

ignore_summaries = {
    "area": ["small", "medium", "large"],
}
coco_eval = naive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    top_k=None,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

Checking GT and Pred files matching...
Checking complete!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.0000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.0000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.0000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.0000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.0000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.0000


In [123]:
coco_gt_file = (
    "/local1/telinwu/research/resources/TREK-150/coco_annotations/trek150_1st_frame_only_narrated_gt.json"
)
coco_pred_file = (
    "/local1/bryanzhou008/jarvis/project_jarvis/baselines"
    "/ego4d-eccv2022-solutions/scod/TREK-150-test-first_frame.json"
)

ignore_summaries = {
    "area": ["small", "medium", "large"],
}
coco_eval = naive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    top_k=1,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

Checking GT and Pred files matching...
Checking complete!
Evaluate annotation type *bbox*
DONE (t=0.04s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=  1 ] = 0.336
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=  1 ] = 0.260
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.361
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.361
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.361


In [285]:
coco_gt_file = (
    "/local1/telinwu/research/resources/TREK-150/coco_annotations"
    "/trek150_all_frames_with_first_mdnet_fail_narrated_gt.json"
)
coco_pred_file = (
    "/local1/bryanzhou008/jarvis/project_jarvis/baselines/ego4d-eccv2022-solutions"
    "/scod/TREK-150-test-all_frame_complete.json"
)

ignore_summaries = {
    "area": ["small", "medium", "large"],
}
coco_eval = naive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    top_k=1,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

Checking GT and Pred files matching...
Checking complete!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.2949
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=  1 ] = 0.4147
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=  1 ] = 0.3085
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.4500
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.4500
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.4500


### Transforming COCO Results to TREK-150 Utilizable Input Dicts

In [286]:
"""
coco_gt_file = (
    "/local1/telinwu/research/resources/TREK-150/coco_annotations"
    "/trek150_all_frames_with_first_mdnet_fail_narrated_gt.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/TREK-150/baselines"
    "/narrated_scod_finetuning_large_v2/eval_ego4d_scod_val_finetuned_45K_on_TREK150_all_frames"
    "/eval/model_0045000/inference/narrated_ego4d_test/bbox.json"
)
coco_pred_file = (
    "/local1/bryanzhou008/jarvis/project_jarvis/baselines/ego4d-eccv2022-solutions/scod/TREK-150-test-1.json"
)
coco_gt_file = (
    "/local1/telinwu/research/resources/TREK-150/paper_data"
    # "/narrated/gpt_v1/gpt_trek150_all_frames_with_first_mdnet_fail_narrated_gt.json"
    # "/narrated/gpt_v2/gpt_trek150_all_frames_with_first_mdnet_fail_narrated_gt.json"
    "/narrated/gt_srl_arg1/trek150_all_frames_with_first_mdnet_fail_narrated_gt.json"
)
"""
coco_gt_file = (
    "/local1/telinwu/research/resources/TREK-150/coco_annotations"
    "/trek150_all_frames_with_first_mdnet_fail_narrated_gt.json"
)
"""
coco_pred_file = (
    "/local1/telinwu/research/resources/TREK-150/baselines"
    # "/ego4d_scod_all_frames_narrated_gpt_v1_srl_arg1_with_tool_drop_null_ooc_tool_symb_conds_mask_no_comma"
    # "/eval_ego4d_scod_val_finetuned_65K_on_TREK150_all_frames"
    # "/eval/model_0065000/inference/narrated_ego4d_test/bbox.json"
    # "/ego4d_scod_all_frames_narrated_gpt_v1_srl_arg1_with_tool_drop_null_ooc_tool_symb_conds_mask_no_comma"
    # "/eval_ego4d_scod_val_finetuned_45K_on_TREK150_all_frames_new"
    # "/eval/model_0045000/inference/narrated_ego4d_test/bbox.json"
    # "/ego4d_scod_all_frames_narrated_gpt_v1_srl_arg1_wo_tool_no_stopwords"
    # "/eval_ego4d_scod_val_finetuned_45K_on_TREK150_all_frames_new"
    # "/eval/model_0045000/inference/narrated_ego4d_test/bbox.json"
    # "/ego4d_scod_all_frames_narrated_full_sentence"
    # "/eval_ego4d_scod_val_finetuned_35K_on_TREK150_stepwise_all_frames"
    # "/eval/model_0035000/inference/narrated_ego4d_test/bbox.json"
    # "/ego4d_scod_all_frames_narrated_gpt_v1_srl_arg1_with_tool_drop_null_ooc_tool_symb_conds_mask_no_comma"
    # "/eval_ego4d_scod_val_finetuned_45K_on_TREK150_all_frames_25percent_prepost"
    # "/eval/model_0045000/inference/narrated_ego4d_test/bbox.json"
    # "/ego4d_scod_all_frames_narrated_gpt_v1_srl_arg1_with_tool_drop_null_ooc_tool_symb_defs"
    # "/eval_ego4d_scod_val_finetuned_45K_on_TREK150_all_frames_25percent_prepost"
    # "/eval/model_0045000/inference/narrated_ego4d_test/bbox.json"
    # "/ego4d_scod_all_frames_narrated_srl_arg1_only"
    # "/eval_ego4d_scod_val_finetuned_55K_on_TREK150_stepwise_all_frames"
    # "/eval/model_0055000/inference/narrated_ego4d_test/bbox.json"
    "/ego4d_scod_all_frames_narrated_gpt_v1_srl_arg1_with_tool_drop_null_ooc_tool_symb_conds_mask_no_comma_defs_new_caps"
    "/eval_ego4d_scod_val_finetuned_35K_on_TREK150_all_frames_25percent_prepost"
    "/eval/model_0035000/inference/narrated_ego4d_test/bbox.json"
)
"""
coco_pred_file = (
    "/local1/bryanzhou008/jarvis/project_jarvis/baselines/ego4d-eccv2022-solutions"
    "/scod/TREK-150-test-all_frame_complete.json"
)

coco_gts = json.load(open(coco_gt_file))
coco_gts_images = coco_gts["images"]
coco_gt_id_mapping = {}
for coco_gt_image in coco_gts_images:
    coco_gt_id_mapping[coco_gt_image["id"]] = coco_gt_image["file_name"]

trek150_results = {}
coco_preds = json.load(open(coco_pred_file))
for coco_pred in tqdm(coco_preds, desc="COCO"):
    image_id = coco_pred["image_id"]
    file_name = coco_gt_id_mapping[image_id]
    video_id = file_name.split("/")[0]
    image_name = file_name.split("/")[-1]
    frame_cnt = image_name.split("_")[-1].split(".")[0]
    frame_cnt_int = int(frame_cnt)
    if video_id not in trek150_results:
        trek150_results[video_id] = {}
    frame_key = frame_cnt
    if frame_key not in trek150_results[video_id]:
        trek150_results[video_id][frame_key] = {
            "bboxes": [],
            "scores": [],
            "scored_bboxes": [],
        }
    trek150_results[video_id][frame_key]["bboxes"].append(coco_pred["bbox"])
    trek150_results[video_id][frame_key]["scores"].append(coco_pred["score"])
    trek150_results[video_id][frame_key]["scored_bboxes"].append((coco_pred["score"], coco_pred["bbox"]))
    
for video_id in trek150_results:
    for frame_key in trek150_results[video_id]:
        trek150_results[video_id][frame_key]["bboxes"] = np.asarray(
            trek150_results[video_id][frame_key]["bboxes"]
        )
        trek150_results[video_id][frame_key]["scores"] = np.asarray(
            trek150_results[video_id][frame_key]["scores"]
        )
        scored_bboxes = sorted(
            trek150_results[video_id][frame_key]["scored_bboxes"],
            reverse=True
        )
        scores = [s for s, b in scored_bboxes]
        bboxes = [b for s, b in scored_bboxes]
        trek150_results[video_id][frame_key]["bboxes"] = np.asarray(bboxes)
        trek150_results[video_id][frame_key]["scores"] = np.asarray(scores)
        del trek150_results[video_id][frame_key]["scored_bboxes"]
# pprint.pprint(trek150_results["P03-P03_02-612"])


"""
json.dump(
    trek150_results,
    open(
        (
            "/local1/telinwu/research/resources/TREK-150/baselines"
            "/narrated_scod_finetuning_large_v2/eval_ego4d_scod_val_finetuned_45K_on_TREK150_all_frames"
            "/eval/model_0045000/inference/narrated_ego4d_test/candidate_boxes.json"
        ),
        "w"
    ),
    indent=4
)
"""

"""
pickle.dump(
    trek150_results,
    open(
        (
            "/local1/telinwu/research/resources/TREK-150/baselines"
            "/narrated_scod_finetuning_large_v2/eval_ego4d_scod_val_finetuned_45K_on_TREK150_all_frames"
            "/eval/model_0045000/inference/narrated_ego4d_test/candidate_boxes.pickle"
        ),
        "wb"
    )
)
pickle.dump(
    trek150_results,
    open(
        (
            "/local1/telinwu/research/resources/TREK-150/baselines"
            "/VideoIntern/eval_on_TREK150_all_frames/candidate_boxes.pickle"
        ),
        "wb"
    )
)
"""
"""
pickle.dump(
    trek150_results,
    open(
        (
            "/local1/telinwu/research/resources/TREK-150/candidate_bboxes"
            # "/GLIP/gpt_v1_symb_conds_mask/candidate_boxes.pickle"
            # "/GLIP/gpt_v2_symb_conds_mask/candidate_boxes.pickle"
            # "/GLIP/gpt_v2_no_symbs/candidate_boxes.pickle"
            # "/GLIP/ground_full_sentence/candidate_boxes.pickle"
            # "/GLIP/gpt_v2_symb_conds_mask_prepost/candidate_boxes.pickle"
            # "/GLIP/gpt_v2_symb_defs/candidate_boxes.pickle"
            # "/GLIP/srl_arg1_only/candidate_boxes.pickle"
            "/GLIP/gpt_v2_symb_conds_mask_prepost_defs/candidate_boxes.pickle"
            
        ),
        "wb"
    )
)
"""
pickle.dump(
    trek150_results,
    open(
        (
            "/local1/telinwu/research/resources/TREK-150/candidate_bboxes"
            "/VideoIntern/candidate_boxes.pickle"
        ),
        "wb"
    )
)

COCO: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 9509400/9509400 [00:48<00:00, 194660.69it/s]


#### Candidatr Bboxes to Run:
**GPT**
* `/local1/telinwu/research/resources/TREK-150/candidate_bboxes/GLIP/gpt_v1_symb_conds_mask/candidate_boxes.pickle`
  * From ego4d **ego4d_scod_all_frames_narrated_gpt_v1_srl_arg1_with_tool_drop_null_ooc_tool_symb_conds_mask_no_comma**
* `/local1/telinwu/research/resources/TREK-150/candidate_bboxes/GLIP/gpt_v2_symb_conds_mask/candidate_boxes.pickle`
  * From ego4d **ego4d_scod_all_frames_narrated_gpt_v1_srl_arg1_with_tool_drop_null_ooc_tool_symb_conds_mask_no_comma**
* `/local1/telinwu/research/resources/TREK-150/candidate_bboxes/GLIP/gpt_v2_no_symbs/candidate_boxes.pickle`
  * From ego4d **ego4d_scod_all_frames_narrated_gpt_v1_srl_arg1_wo_tool_no_stopwords**
* `/local1/telinwu/research/resources/TREK-150/candidate_bboxes/GLIP/gpt_v2_symb_conds_mask_prepost/candidate_boxes.pickle`
  * From ego4d **ego4d_scod_all_frames_narrated_gpt_v1_srl_arg1_with_tool_drop_null_ooc_tool_symb_conds_mask_no_comma**
  * Really added the pre/post conditional statements (prev ones forgot...)
* `/local1/telinwu/research/resources/TREK-150/candidate_bboxes/GLIP/gpt_v2_symb_defs/candidate_boxes.pickle`
  * From ego4d **ego4d_scod_all_frames_narrated_gpt_v1_srl_arg1_with_tool_drop_null_ooc_tool_symb_defs**
* `/local1/telinwu/research/resources/TREK-150/candidate_bboxes/GLIP/gpt_v2_symb_conds_mask_prepost_defs/candidate_boxes.pickle`
  * From ego4d **ego4d_scod_all_frames_narrated_gpt_v1_srl_arg1_with_tool_drop_null_ooc_tool_symb_conds_mask_no_comma_defs_new_caps**
* `path`
  * File description?
  
**Baselines**
* `/local1/telinwu/research/resources/TREK-150/candidate_bboxes/GLIP/ground_full_sentence/candidate_boxes.pickle`
  * From ego4d **ego4d_scod_all_frames_narrated_full_sentence**
* `/local1/telinwu/research/resources/TREK-150/candidate_bboxes/GLIP/gt_srl_arg1/candidate_boxes.pickle `
  * From ego4d **narrated_scod_finetuning_large_v2** (Basically **gt-srl-arg1**)
* `/local1/telinwu/research/resources/TREK-150/candidate_bboxes/VideoIntern/candidate_boxes.pickle`
  * **VideoIntern-L** baseline
* `/local1/telinwu/research/resources/TREK-150/candidate_bboxes/GLIP/srl_arg1_only/candidate_boxes.pickle`
  * From ego4d **ego4d_scod_all_frames_narrated_srl_arg1_only**
* `path`
  * File description?

In [172]:
"""
class GPTSymbolic(object):
    def __init__(self, ...):
        self.gpt_instance = ...
        # other params ...
    
    def _state_change(self, sent):
        prompt = "... {} ...".format(sent)
        response = self.gpt_instance(prompt)
        return response
    
    def _spatial(self, sent):
        prompt = "... {} ...".format(sent)
        response = self.gpt_instance(prompt)
        return response
    
    # More classes
    
    def get_gpt_response(self, sent, mode=None):
        if mode == "spatial":
            return self._spatial(sent)
        # elif ...
        
    def pipeline(self, sent):
        ooc, tool = self.get_gpt_response(sent, mode="obj_type")
        spatials = self.get_gpt_response(sent, mode="spatial")
        # ...
        results = {
            "gpt_responses": {
                "obj_type": "...",
                "spatial": "..."
                # ...
            }
            "parsed_responses": {
                "spatials": "...",
                "state_changes": "...",
            }
        }
        return results
"""
print("NA")

NA


# With Narrations

In [12]:
def print_ek_video_narrations(
    video_id,
    epic_kitchens_data,
    verbose=False,
):
    frame_narration_list = []
    
    if verbose:
        print("Video ID: {}".format(video_id))
    video = epic_kitchens_data[video_id]

    event_id = 1
    main_verbs = []
    main_nouns = []
    for event in video:
        all_verbs, all_nouns = event["verb"], event["all_nouns"]
        main_verbs += [all_verbs]
        main_nouns += all_nouns
    main_verbs = sorted(list(set(main_verbs)))
    main_nouns = sorted(list(set(main_nouns)))
    if verbose:
        print("Main Verbs: {}".format(main_verbs))
        print("Main Nouns: {}".format(main_nouns))
    prev_stop_time = None
    for event in video:
        start_timestamp, stop_timestamp = event["start_timestamp"], event["stop_timestamp"]
        start_frame, stop_frame = int(event["start_frame"]), int(event["stop_frame"])
        narration = event["narration"]
        t = datetime.strptime(start_timestamp, "%H:%M:%S.%f")
        delta = timedelta(hours=t.hour, minutes=t.minute, seconds=t.second, microseconds=t.microsecond)
        curr_start_time = delta.total_seconds()
        t = datetime.strptime(stop_timestamp, "%H:%M:%S.%f")
        delta = timedelta(hours=t.hour, minutes=t.minute, seconds=t.second, microseconds=t.microsecond)
        curr_stop_time = delta.total_seconds()
        if verbose:
            if prev_stop_time is not None and curr_start_time < prev_stop_time:
                print("({:04d}) [{}-{}] {} (↑)".format(event_id, start_timestamp, stop_timestamp, narration))
            else:
                print("({:04d}) [{}-{}] {} ".format(event_id, start_timestamp, stop_timestamp, narration))
        event_id += 1
        prev_stop_time = curr_stop_time
        frame_narration_list.append((start_frame, stop_frame, (start_frame+stop_frame)//2, narration))
    
    return frame_narration_list

In [13]:
epic_kitchens_root = "/local1/telinwu/research/resources/EPIC-KITCHENS-100/"
epic_kitchens_train_file = "EPIC_100_train.csv"
epic_kitchens_train_file = os.path.join(epic_kitchens_root, "epic-kitchens-100-annotations", epic_kitchens_train_file)
epic_kitchens_csv = csv.DictReader(open(epic_kitchens_train_file))

In [52]:
action_class_mappings = {
    "verb": {},
    "noun": {},
}

class_mapping_csv =  {
    "verb": "/local1/bryanzhou008/jarvis/epic_kitchen/epic-kitchens-55-annotations/EPIC_verb_classes.csv",
    "noun": "/local1/bryanzhou008/jarvis/epic_kitchen/epic-kitchens-55-annotations/EPIC_noun_classes.csv",
}
verb_class_mapping_csv = "/local1/bryanzhou008/jarvis/epic_kitchen/epic-kitchens-55-annotations/EPIC_verb_classes.csv"

for class_type in class_mapping_csv:
    csv_reader = csv.DictReader(open(class_mapping_csv[class_type], "r"))
    for row in csv_reader:
        class_id = row["{}_id".format(class_type)]
        class_key = row["class_key"]
        allowed = ast.literal_eval(row["{}s".format(class_type)])
        allowed = [x.split(":")[-1] for x in allowed]
        action_class_mappings[class_type][int(class_id)] = [class_key] + allowed
pass

pprint.pprint(action_class_mappings["noun"])

{0: ['Nothing', 'Nothing'],
 1: ['pan',
     'pan',
     'sauce',
     'frying',
     'cake',
     'saucepan',
     'empty',
     'small',
     'wok',
     'scour',
     'content'],
 2: ['pan:dust', 'dust', 'dustpan'],
 3: ['tap', 'tap', 'water', 'nozzle', 'coffee'],
 4: ['plate',
     'plate',
     'pasta',
     'saucer',
     'dish',
     'salad',
     'lasagna',
     'plate',
     'flatware',
     'pate'],
 5: ['knife',
     'knife',
     'mincing',
     'using',
     'blade',
     'chopper',
     'mezzaluna',
     'moon',
     'mezzaluna'],
 6: ['bowl',
     'bowl',
     'salad',
     'glass',
     'sugar',
     'empty',
     'cooker',
     'washing',
     'egg',
     'processor'],
 7: ['spoon',
     'spoon',
     'wooden',
     'using',
     'teaspoon',
     'spoon',
     'spade',
     'food',
     'coffee',
     'tea',
     'measure',
     'coffee',
     'scoop',
     'stirrer',
     'tablespoon'],
 8: ['cupboard',
     'cupboard',
     'cabinet',
     'locker',
     'flap',
    

In [14]:
epic_kitchens_data = {}

for row in epic_kitchens_csv:
    video_id = row["video_id"]
    if video_id not in epic_kitchens_data:
        epic_kitchens_data[video_id] = []
    
    curr_timestamp = {}
    for key in row:
        if key in ["video_id", "participant_id", "narration_id", "narration_timestamp"]:
            continue
        if key in ["all_nouns", "all_noun_classes"]:
            row[key] = literal_eval(row[key])
        curr_timestamp[key] = row[key]
    epic_kitchens_data[video_id].append(curr_timestamp)
    
for video_id in epic_kitchens_data:
    epic_kitchens_data[video_id] = sorted(
        epic_kitchens_data[video_id],
        key=lambda x: int(x["start_frame"])
    )

In [53]:
video_id = "P03-P03_02-612"
video_id = "P03-P03_04-57"
ek_video_id = video_id.split("-")[1]

frame_narration_list = print_ek_video_narrations(
    ek_video_id,
    epic_kitchens_data,
    verbose=False,
)

print(frame_narration_list[:5])

[(194, 486, 340, 'put bags'), (390, 454, 422, 'take squash'), (454, 515, 484, 'put squash'), (537, 588, 562, 'take salmon'), (583, 635, 609, 'put salmon')]


In [144]:
# Inits.
image_id_accum = 0
annot_id_accum = 0
# frame_num_step = 10
frame_num_step = 1
coco_format_images = []
coco_format_annots = []


# Testing start frame dict.
trek150_data_root = "/local1/bryanzhou008/jarvis/epic_kitchen/clean/fpv-tracking-baselines/LTMU-H/TREK-150"
start_frames_dict = json.load(open(
    "/local1/bryanzhou008/jarvis/epic_kitchen/clean/fpv-tracking-baselines"
    "/LTMU-H/TREK-150-toolkit/first_mistakes.json"
))
per_video_start_frames = {}
for video_id in video_ids:
    if video_id not in start_frames_dict:
        per_video_start_frames[video_id] = None
    else:
        per_video_start_frames[video_id] = int(start_frames_dict[video_id])
########
per_video_start_frames = None


for video_id in tqdm(video_ids, desc="Processing TREK-150 Videos..."):
    ek_video_id = video_id.split("-")[1]
    frame_narration_list = print_ek_video_narrations(
        ek_video_id,
        epic_kitchens_data,
        verbose=False,
    )
    
    # if video_id == "P03-P03_04-57":
    if video_id == "P03-P03_04-48":
        # verbose = True
        verbose = False
    else:
        verbose = False
    
    image_id_accum, annot_id_accum = process_one_trek150_video(
        video_id=video_id,
        trek150_data_root="/local1/bryanzhou008/jarvis/epic_kitchen/clean/fpv-tracking-baselines/LTMU-H/TREK-150",
        frame_num_step=frame_num_step,
        image_id_accum=image_id_accum,
        annot_id_accum=annot_id_accum,
        coco_format_images=coco_format_images,
        coco_format_annots=coco_format_annots,
        narrations_info_dict=frame_narration_list,
        token_positive_method="gt",
        action_class_mappings=action_class_mappings,
        per_video_start_frames=per_video_start_frames,
        verbose=verbose,
    )

print(len(coco_format_images), len(coco_format_annots))

save_coco_file(
    coco_format_images,
    coco_format_annots,
    output_root="/local1/telinwu/research/resources/TREK-150/coco_annotations",
    # output_name="trek150_stepwise_10_frames_narrated_gt",
    output_name="trek150_all_frames_with_first_mdnet_fail_narrated_gt",
)

Processing TREK-150 Videos...:   3%|██▊                                                                                | 5/150 [00:00<00:05, 28.20it/s]

turn on tap
['pan', 'pan', 'sauce', 'frying', 'cake', 'saucepan', 'empty', 'small', 'wok', 'scour', 'content']
[11, 3, 1]
22440 22690
[(22391, 22597, 'turn on tap'),
 (22663, 22707, 'turn off tap'),
 (22307, 22358, 'push bin back'),
 (22100, 22301, 'put lid on bin'),
 (21801, 22121, 'open bag')]
P03-P03_02-56
dry pan
[[4, 7]]
--------------------------------------------------


Processing TREK-150 Videos...:  20%|████████████████▍                                                                 | 30/150 [00:01<00:05, 22.60it/s]

take plate
['jug', 'jug', 'measuring', 'pitcher', 'carafe', 'rinse']
[0, 4, 105]
21741 22090
[(21829, 21993, 'take plate'),
 (21999, 22144, 'take packet of cookies'),
 (21648, 21781, 'take packets of cookies'),
 (21458, 21543, 'open microwave'),
 (21208, 21452, 'place cup on table')]
P05-P05_03-21
take jug
[[5, 8]]
--------------------------------------------------


Processing TREK-150 Videos...:  27%|█████████████████████▊                                                            | 40/150 [00:01<00:04, 26.81it/s]

...

closing the door
['door:kitchen', 'kitchen']
[3, 34, 34]
17826 18040
[(17787, 17963, 'closing the door'),
 (15387, 17781, 'cut the onion into the small piece'),
 (18199, 18349, 'opening the door'),
 (18382, 18489, 'adjust the chair'),
 (18920, 18967, 'open the tap')]
P23-P23_03-243
close door:kitchen
[[6, 18]]
--------------------------------------------------
clean the bowl
['colander', 'colander', 'pasta']
[4, 42, 42]
91491 92770
[(91586, 92680, 'clean the bowl'),
 (92623, 92769, 'cleaning the bowl'),
 (88230, 91309, 'mixing the curry'),
 (92804, 92853, 'take the brush'),
 (93851, 94101, 'put the dust into dustbin')]
P23-P23_04-389
wash colander
[[5, 13]]
--------------------------------------------------


Processing TREK-150 Videos...:  82%|██████████████████████████████████████████████████████████████████▍              | 123/150 [00:06<00:01, 20.53it/s]

put on detergent
['sponge', 'sponge', 'scrubber']
[4, 21, 21]
36291 36700
[(36390, 36622, 'put on detergent'),
 (36291, 36385, 'put down knife'),
 (35680, 36285, 'clean knife'),
 (34887, 35625, 'put on detergent'),
 (35163, 35390, 'rinse sponge')]
P25-P25_09-2061
wash sponge
[[5, 11]]
--------------------------------------------------


Processing TREK-150 Videos...: 100%|█████████████████████████████████████████████████████████████████████████████████| 150/150 [00:07<00:00, 19.60it/s]


95094 95094
Saving coco file to: /local1/telinwu/research/resources/TREK-150/coco_annotations/trek150_all_frames_with_first_mdnet_fail_narrated_gt.json


In [111]:
# Inits.
image_id_accum = 0
annot_id_accum = 0
frame_num_step = 100000000
coco_format_images = []
coco_format_annots = []
per_video_start_frames = None


for video_id in tqdm(video_ids, desc="Processing TREK-150 Videos..."):
    ek_video_id = video_id.split("-")[1]
    frame_narration_list = print_ek_video_narrations(
        ek_video_id,
        epic_kitchens_data,
        verbose=False,
    )
    
    # if video_id == "P03-P03_04-57":
    if video_id == "P03-P03_04-48":
        # verbose = True
        verbose = False
    else:
        verbose = False
    
    image_id_accum, annot_id_accum = process_one_trek150_video(
        video_id=video_id,
        trek150_data_root="/local1/bryanzhou008/jarvis/epic_kitchen/clean/fpv-tracking-baselines/LTMU-H/TREK-150",
        frame_num_step=frame_num_step,
        image_id_accum=image_id_accum,
        annot_id_accum=annot_id_accum,
        coco_format_images=coco_format_images,
        coco_format_annots=coco_format_annots,
        narrations_info_dict=frame_narration_list,
        token_positive_method="gt",
        action_class_mappings=action_class_mappings,
        per_video_start_frames=per_video_start_frames,
        verbose=verbose,
    )

print(len(coco_format_images), len(coco_format_annots))

save_coco_file(
    coco_format_images,
    coco_format_annots,
    output_root="/local1/telinwu/research/resources/TREK-150/coco_annotations",
    output_name="trek150_1st_frame_only_narrated_gt",
)

Processing TREK-150 Videos...:  46%|█████████████████████████████████████▎                                           | 69/150 [00:00<00:00, 345.44it/s]

turn on tap
['pan', 'pan', 'sauce', 'frying', 'cake', 'saucepan', 'empty', 'small', 'wok', 'scour', 'content']
[11, 3, 1]
22440 22690
[(22391, 22597, 'turn on tap'),
 (22663, 22707, 'turn off tap'),
 (22307, 22358, 'push bin back'),
 (22100, 22301, 'put lid on bin'),
 (21801, 22121, 'open bag')]
P03-P03_02-56
dry pan
[[4, 7]]
--------------------------------------------------
take plate
['jug', 'jug', 'measuring', 'pitcher', 'carafe', 'rinse']
[0, 4, 105]
21741 22090
[(21829, 21993, 'take plate'),
 (21999, 22144, 'take packet of cookies'),
 (21648, 21781, 'take packets of cookies'),
 (21458, 21543, 'open microwave'),
 (21208, 21452, 'place cup on table')]
P05-P05_03-21
take jug
[[5, 8]]
--------------------------------------------------
wash spoon
['sponge', 'sponge', 'scrubber']
[4, 7, 21]
54571 55176
[(54615, 55185, 'wash spoon'),
 (54519, 54561, 'take spoon'),
 (53712, 54409, 'wash wooden spoon'),
 (53610, 53666, 'take wooden spoon'),
 (51853, 53366, 'wash pot')]
P05-P05_03-4
wash s

Processing TREK-150 Videos...: 100%|████████████████████████████████████████████████████████████████████████████████| 150/150 [00:00<00:00, 382.25it/s]

wash the dish
['box', 'box', 'lunch', 'cereal', 'tea', 'salt', 'tupperware', 'lunchbox', 'case', 'mushroom', 'coffee', 'cheese', 'milk', 'rice', 'carrot', 'fruit', 'plastic', 'tub', 'plastic', 'eggs', 'fajita']
[4, 39, 39]
3591 5830
[(3569, 6207, 'wash the dish'),
 (3762, 4104, 'wash the dish'),
 (2888, 3675, 'wash the dish'),
 (2332, 2773, 'wash the chopsticks'),
 (1080, 2347, 'wash the pan')]
P16-P16_03-151
wash box
[[5, 8]]
--------------------------------------------------
put chicken into bowl
['meat', 'meat', 'minced', 'meat', 'meat', 'mix', 'mincemeat', 'mince']
[0, 26, 26]
31761 32246
[(31857, 32310, 'put chicken into bowl'),
 (31319, 31744, 'pour water'),
 (29628, 31259, 'still heat bacon'),
 (32326, 32424, 'put chicken into bowl'),
 (29604, 29856, 'still heat bacon')]
P19-P19_04-148
take meat
[[5, 9]]
--------------------------------------------------
wash chicken
['meat', 'meat', 'minced', 'meat', 'meat', 'mix', 'mincemeat', 'mince']
[24, 23, 26]
15861 16540
[(15956, 16454, 

# Distribute GPT Results to Other Files

In [216]:
import copy

In [225]:
init_gpt_file = (
    "/local1/telinwu/research/resources/TREK-150/paper_data/narrated"
    # "/gpt_v1/gpt_trek150_1st_frame_only_narrated_gt.json"
    "/gpt_v2/gpt_trek150_1st_frame_only_narrated_gt.json"
)

files_to_insert = [
    (
        "/local1/telinwu/research/resources/TREK-150/paper_data/narrated"
        "/gt_srl_arg1/trek150_stepwise_10_frames_narrated_gt.json"
    ),
    (
        "/local1/telinwu/research/resources/TREK-150/paper_data/narrated"
        "/gt_srl_arg1/trek150_all_frames_with_first_mdnet_fail_narrated_gt.json"
    ),
]

gpt_data = json.load(open(init_gpt_file))
gpt_symb_dict = {}
for image in gpt_data["images"]:
    file_name = image["file_name"]
    video_uid = file_name.split("/")[0]
    gpt_symb_dict[video_uid] = image

gpt_file_folder = "/".join(init_gpt_file.split("/")[:-1])
folder_suffix = None
folder_suffix = "25percent_prepost"

if folder_suffix is not None:
    gpt_file_folder += "_{}".format(folder_suffix)
if not os.path.exists(gpt_file_folder):
    os.makedirs(gpt_file_folder)

for f in files_to_insert:
    file_name = f.split("/")[-1]
    new_file_name = "gpt_" + file_name
    new_file_path = os.path.join(gpt_file_folder, new_file_name)
    
    trek_data = json.load(open(f))
    trek_images = trek_data["images"]
    
    per_video_frames = {}
    for image in trek_images:
        file_name = image["file_name"]
        video_uid = file_name.split("/")[0]
        video_frame_num = int(file_name.split("/")[-1].split("_")[-1].split(".")[0])
        if video_uid not in per_video_frames:
            per_video_frames[video_uid] = {}
        per_video_frames[video_uid][video_frame_num] = image
    
    for video_uid in per_video_frames:
        frame_nums = sorted(list(per_video_frames[video_uid].keys()))
        frames_diff = frame_nums[-1] - frame_nums[0]
        first_K_pre = 10
        last_K_post = 10
        first_K_pre = int(frames_diff * 0.25)
        last_K_post = int(frames_diff * 0.25)
        for frame_num in frame_nums:
            if frame_num <= frame_nums[0] + first_K_pre:
                current_frame_type = "pre"
            elif frame_num >= frame_nums[-1] - last_K_post:
                current_frame_type = "post"
            else:
                current_frame_type = "pnr"
            gpt_image = copy.deepcopy(gpt_symb_dict[video_uid])
            if "symbolic" not in gpt_image:
                print(video_uid)
            symb = gpt_image["symbolic"]
            symb["current_frame_type"] = current_frame_type
            curr_image = per_video_frames[video_uid][frame_num]
            curr_image["symbolic"] = symb
            curr_image["caption"] = gpt_image["caption"]
            curr_image["tokens_positive_eval"] = gpt_image["tokens_positive_eval"]
            curr_image["gpt_pass"] = gpt_image["gpt_pass"]
        pass
    
    images_to_insert = []
    for video_uid in per_video_frames:
        frame_nums = sorted(list(per_video_frames[video_uid].keys()))
        for frame_num in frame_nums:
            images_to_insert.append(per_video_frames[video_uid][frame_num])
    images_to_insert = sorted(images_to_insert, key=lambda x: x["id"])
    
    trek_data["images"] = images_to_insert
    json.dump(
        trek_data,
        open(new_file_path, "w"),
        indent=4,
    )
    
    print("Saving to: {}".format(new_file_path))

Saving to: /local1/telinwu/research/resources/TREK-150/paper_data/narrated/gpt_v2_25percent_prepost/gpt_trek150_stepwise_10_frames_narrated_gt.json
Saving to: /local1/telinwu/research/resources/TREK-150/paper_data/narrated/gpt_v2_25percent_prepost/gpt_trek150_all_frames_with_first_mdnet_fail_narrated_gt.json


### For Full Sentence

In [ ]:
init_ref_file = (
    "/local1/telinwu/research/resources/TREK-150/paper_data"
    "/narrated/gt_srl_arg1/trek150_1st_frame_only_narrated_gt.json"
)

output_file = init_ref_file.replace("gt_srl_arg1", "full_sentence").replace("_gt.json", "_full_sentence.json")

init_data = json.load(open(init_ref_file))
for image_idx in range(len(init_data["images"])):
    caption = init_data["images"][image_idx]["caption"]
    new_tokens_positive_eval = [[[0, len(caption)]]]
    init_data["images"][image_idx]["tokens_positive_eval"] = new_tokens_positive_eval
json.dump(
    init_data,
    open(output_file, "w"),
    indent=4,
)
print("Saving to: {}".format(output_file))

init_new_file = output_file
init_new_data = json.load(open(init_new_file))
init_data_dict = {}
for image in init_new_data["images"]:
    file_name = image["file_name"]
    video_uid = file_name.split("/")[0]
    init_data_dict[video_uid] = image

files_to_insert = [
    (
        "/local1/telinwu/research/resources/TREK-150/paper_data/narrated"
        "/gt_srl_arg1/trek150_stepwise_10_frames_narrated_gt.json"
    ),
    (
        "/local1/telinwu/research/resources/TREK-150/paper_data/narrated"
        "/gt_srl_arg1/trek150_all_frames_with_first_mdnet_fail_narrated_gt.json"
    ),
]

file_folder = "/".join(init_new_file.split("/")[:-1])

for f in files_to_insert:
    file_name = f.split("/")[-1]
    new_file_name = file_name.replace("_gt.json", "_full_sentence.json")
    new_file_path = os.path.join(file_folder, new_file_name)
    
    trek_data = json.load(open(f))
    trek_images = trek_data["images"]
    
    per_video_frames = {}
    for image in trek_images:
        file_name = image["file_name"]
        video_uid = file_name.split("/")[0]
        video_frame_num = int(file_name.split("/")[-1].split("_")[-1].split(".")[0])
        if video_uid not in per_video_frames:
            per_video_frames[video_uid] = {}
        per_video_frames[video_uid][video_frame_num] = image
    
    for video_uid in per_video_frames:
        frame_nums = sorted(list(per_video_frames[video_uid].keys()))
        for frame_num in frame_nums:
            curr_image = per_video_frames[video_uid][frame_num]
            curr_image["tokens_positive_eval"] = init_data_dict[video_uid]["tokens_positive_eval"]
        pass
    
    images_to_insert = []
    for video_uid in per_video_frames:
        frame_nums = sorted(list(per_video_frames[video_uid].keys()))
        for frame_num in frame_nums:
            images_to_insert.append(per_video_frames[video_uid][frame_num])
    images_to_insert = sorted(images_to_insert, key=lambda x: x["id"])
    
    trek_data["images"] = images_to_insert
    json.dump(
        trek_data,
        open(new_file_path, "w"),
        indent=4,
    )
    
    print("Saving to: {}".format(new_file_path))

# Make SCOD Tracking (Use `PNR` to **track** `Post`)

In [240]:
coco_gt_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/paper_data/all_frames/narrated"
    "/gpt_v3_srl_arg1_with_tool/val_scod_all_frames_narrated_gpt_v3_srl_arg1_with_tool.json"
)
coco_pred_file = (
    "/local1/telinwu/research/resources/Ego4D/ego4d_scod_challenge/paper_models/all_frames/narrated"
    "/gpt_v3_srl_arg1_with_tool"
    "/ego4d_scod_all_frames_narrated_gpt_v3_srl_arg1_with_tool_drop_null_ooc_tool_symb_conds_mask_no_comma"
    "/eval_at_45K/eval/model_0045000/inference/narrated_ego4d_test/bbox.json"
)


coco_gt_data = json.load(open(coco_gt_file))
coco_pred_data = json.load(open(coco_pred_file))

coco_gt_id_mapping = {}
for coco_gt_image in coco_gt_data["images"]:
    coco_gt_id_mapping[coco_gt_image["id"]] = coco_gt_image["file_name"]

In [245]:
trek150_results = {}
coco_preds = json.load(open(coco_pred_file))
for coco_pred in tqdm(coco_preds, desc="COCO"):
    image_id = coco_pred["image_id"]
    file_name = coco_gt_id_mapping[image_id]
    video_id = file_name.split("/")[0]
    image_name = file_name.split("/")[-1]
    frame_cnt = image_name.split("_")[-1].split(".")[0]
    frame_cnt_int = int(frame_cnt)
    if video_id not in trek150_results:
        trek150_results[video_id] = {}
    frame_key = frame_cnt
    if frame_key not in trek150_results[video_id]:
        trek150_results[video_id][frame_key] = {
            "bboxes": [],
            "scores": [],
            "scored_bboxes": [],
        }
    trek150_results[video_id][frame_key]["bboxes"].append(coco_pred["bbox"])
    trek150_results[video_id][frame_key]["scores"].append(coco_pred["score"])
    trek150_results[video_id][frame_key]["scored_bboxes"].append((coco_pred["score"], coco_pred["bbox"]))
    
for video_id in trek150_results:
    for frame_key in trek150_results[video_id]:
        trek150_results[video_id][frame_key]["bboxes"] = np.asarray(
            trek150_results[video_id][frame_key]["bboxes"]
        )
        trek150_results[video_id][frame_key]["scores"] = np.asarray(
            trek150_results[video_id][frame_key]["scores"]
        )
        scored_bboxes = sorted(
            trek150_results[video_id][frame_key]["scored_bboxes"],
            reverse=True
        )
        scores = [s for s, b in scored_bboxes]
        bboxes = [b for s, b in scored_bboxes]
        trek150_results[video_id][frame_key]["bboxes"] = np.asarray(bboxes)
        trek150_results[video_id][frame_key]["scores"] = np.asarray(scores)
        del trek150_results[video_id][frame_key]["scored_bboxes"]

pickle.dump(
    trek150_results,
    open(
        (
            "/local1/telinwu/research/resources/TREK-150/paper_data/scod_track"
            "/use_pnr_to_track_post/candidate_boxes.pickle"
        ),
        "wb"
    )
)

COCO: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 1070544/1070544 [00:09<00:00, 117220.16it/s]


In [262]:
scod_track_root_folder = "/local1/telinwu/research/resources/TREK-150/paper_data/scod_track/use_pnr_to_track_post"
clip_folder = os.path.join(scod_track_root_folder, "clips")

image_id_annots_dict = {}
for annot in coco_gt_data["annotations"]:
    image_id = annot["image_id"]
    if image_id not in image_id_annots_dict:
        image_id_annots_dict[image_id] = []
    image_id_annots_dict[image_id].append(annot)

pnr_and_post_images = []
curr_tuple = []
for image in coco_gt_data["images"]:
    ego4d_scod_id = image["ego4d_scod_id"]
    if "pre" in ego4d_scod_id:
        if len(curr_tuple) > 0:
            pnr_and_post_images.append(curr_tuple)
        curr_tuple = []
        continue
    if "pnr" in ego4d_scod_id:
        curr_tuple.append(image)
    if (
        "post" in ego4d_scod_id
        and "pnr" in curr_tuple[-1]["ego4d_scod_id"]
    ):
        curr_tuple.append(image)

clip_folder_names = []
for frame_tuple in tqdm(pnr_and_post_images, desc="Tracking"):
    assert len(frame_tuple) == 2
    pnr_frame, post_frame = frame_tuple
    pnr_frame_scod_id, post_frame_scod_id = pnr_frame["ego4d_scod_id"], post_frame["ego4d_scod_id"]
    pnr_image_id, post_image_id = pnr_frame["id"], post_frame["id"]
    if (
        pnr_image_id not in image_id_annots_dict
        or
        post_image_id not in image_id_annots_dict
    ):
        continue
    pnr_annot, post_annot = image_id_annots_dict[pnr_image_id], image_id_annots_dict[post_image_id]
    pnr_annot =  [x for x in pnr_annot if x["category_id"] == 1]
    post_annot = [x for x in post_annot if x["category_id"] == 1]
    if (
        len(pnr_annot) <= 0
        or
        len(post_annot) <= 0
    ):
        continue
    pnr_annot, post_annot = pnr_annot[0], post_annot[0]
    pnr_gt_bbox  = pnr_annot["bbox"]
    post_gt_bbox = post_annot["bbox"]
    pnr_frame_name  = pnr_frame["file_name"]
    post_frame_name = post_frame["file_name"]
    
    video_uid = pnr_frame_name.split("/")[0]
    pnr_frame_num =  pnr_frame_name.split("/")[-1].split(".")[0]
    post_frame_num = post_frame_name.split("/")[-1].split(".")[0]
    clip_folder_name = "{}_pnr_{}_post_{}".format(video_uid, pnr_frame_num, post_frame_num)
    clip_folder_path = os.path.join(clip_folder, clip_folder_name)
    if os.path.exists(clip_folder_path):
        shutil.rmtree(clip_folder_path)
    os.makedirs(clip_folder_path)
    clip_folder_names.append(clip_folder_name)
    
    groundtruth_rects = [
        ",".join([str(x) for x in pnr_gt_bbox]),
        ",".join([str(x) for x in post_gt_bbox]),
    ]
    groundtruth_rects_file = open(os.path.join(clip_folder_path, "groundtruth_rect.txt"), "w")
    for groundtruth_rect in groundtruth_rects:
        groundtruth_rects_file.write(groundtruth_rect+"\n")
    groundtruth_rects_file.close()
    
    frame_names = [
        pnr_frame_name,
        post_frame_name,
    ]
    frame_file = open(os.path.join(clip_folder_path, "frames.txt"), "w")
    for frame_name in frame_names:
        frame_file.write(frame_name+"\n")
    frame_file.close()

sequences_file = open(os.path.join(scod_track_root_folder, "sequences.txt"), "w")
for clip_folder_name in clip_folder_names:
    sequences_file.write(clip_folder_name+"\n")
sequences_file.close()

Tracking: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 12800/12800 [00:01<00:00, 7402.63it/s]


## Performance

In [299]:
def get_ndarray_coco_results(coco_pred_file):
    # [Nx7] where each row contains {imageID,x1,y1,w,h,score,class}
    if type(coco_pred_file) == str:
        preds = json.load(open(coco_pred_file))
    else:
        preds = coco_pred_file
    arrs = []
    for pred in preds:
        image_id = pred["image_id"]
        x1 = pred["bbox"][0]
        y1 = pred["bbox"][1]
        w  = pred["bbox"][2]
        h  = pred["bbox"][3]
        score = pred["score"]
        label = pred["category_id"]
        arr = [image_id, x1, y1, w, h, score, label]
        arrs.append(arr)
    return np.asarray(arrs)


def naive_ego4d_scod_coco_results(
    coco_gt_file,
    coco_pred_file,
    original_coco_gt_file=None,
    top_k=None,
    ignore_summaries=None,
    do_not_summarize=False,
    verbose=False,
):  
    if original_coco_gt_file is not None:
        coco_gt = COCO(annotation_file=original_coco_gt_file, verbose=verbose)
        coco_gt = coco_gt.loadRes(coco_gt_file)
        check_gt_pred_match_on_file(original_coco_gt_file, coco_pred_file)
    else:
        coco_gt = COCO(annotation_file=coco_gt_file, verbose=verbose)
        check_gt_pred_match_on_file(coco_gt_file, coco_pred_file)
    coco_dt = coco_gt.loadRes(coco_pred_file)
    coco_eval = COCOeval(coco_gt, coco_dt, "bbox")
    coco_eval.params.verbose = verbose
    if top_k is not None:
        coco_eval.params.maxDets = [top_k, top_k, top_k]
    coco_eval.evaluate()
    coco_eval.accumulate()
    if not do_not_summarize:
        coco_eval.summarize(ignore_summaries=ignore_summaries)
        perfs = coco_eval.stats
    return coco_eval, perfs

In [309]:
sequences_file = (
    # "/local1/bryanzhou008/jarvis/epic_kitchen/clean/fpv-tracking-baselines/LTMU-H/TREK-150/sequences.txt"
    "/local1/telinwu/research/resources/TREK-150/paper_data/scod_track/use_pnr_to_track_post/sequences.txt"
)
results_folder = (
    # "/local1/bryanzhou008/jarvis/epic_kitchen/clean/fpv-tracking-baselines"
    # "/LTMU-H/results_scores/gpt_v2_symb_conds+defs/ope"
    "/local1/bryanzhou008/jarvis/epic_kitchen/clean/fpv-tracking-baselines"
    "/LTMU-H/results_scores_scod/gpt_v2_use_pnr_to_track_post_oped/ope"
    # "/LTMU-H/results_scores_scod/gpt_v2_use_pnr_to_track_post/ope"
)
gt_folder = (
    # "/local1/bryanzhou008/jarvis/epic_kitchen/clean/fpv-tracking-baselines/LTMU-H/TREK-150/"
    "/local1/telinwu/research/resources/TREK-150/paper_data/scod_track/use_pnr_to_track_post/clips"
)

video_ids = open(sequences_file).readlines()
video_ids = [s.strip() for s in video_ids]

annot_idx = 0
annotations = []
images = []
all_preds = []
categories = [
    {
        "id": 1,
        "name": "object_of_change", 
        "supercategory": "object",
    }
]

for video_id in video_ids:
    res_file_path = os.path.join(results_folder, "{}.txt".format(video_id))
    if not os.path.exists(res_file_path):
        continue
    pred_bboxes = open(res_file_path).readlines()
    pred_bboxes = [x.strip() for x in pred_bboxes][1:]
    
    gt_file = os.path.join(gt_folder, video_id, "groundtruth_rect.txt")
    
    gt_bboxes = open(gt_file).readlines()
    gt_bboxes = [x.strip() for x in gt_bboxes][1:]
    
    for idx in range(len(gt_bboxes)):
        gt_bbox = gt_bboxes[idx]
        gt_bbox = [float(x) for x in gt_bbox.split(",")]
        pred_bbox = pred_bboxes[idx]
        pred_bbox = [float(x) for x in pred_bbox.split(",")]
        
        images.append({
            "file_name": "xxx.jpg",
            "height": 1080,
            "width": 1920,
            "id": annot_idx,
        })
        
        annotations.append({
            "area": gt_bbox[2] * gt_bbox[3],
            "iscrowd": 0,
            "ignore": 0,
            "bbox": gt_bbox,
            "image_id": annot_idx,
            "category_id": 1,
            "id": annot_idx,
        })

        pred_dict = {
            "image_id": annot_idx,
            "category_id": 1,
            "bbox": pred_bbox,
            "score": 1.0,
        }
        all_preds.append(pred_dict)
        annot_idx += 1
    pass

gt_data = {
    "categories": categories,
    "images": images,
    "annotations": annotations,
}

json.dump(
    gt_data,
    open("./tmp/trek_tmp_gt.json", "w"),
    indent=4,
)

json.dump(
    all_preds,
    open("./tmp/trek_tmp_pred.json", "w"),
    indent=4,
)

In [310]:
coco_gt_file = (
    "./tmp/trek_tmp_gt.json"
)
coco_pred_file = (
    "./tmp/trek_tmp_pred.json"
)

ignore_summaries = {
    "area": ["small", "medium", "large"],
}
coco_eval = naive_ego4d_scod_coco_results(
    coco_gt_file=coco_gt_file,
    coco_pred_file=coco_pred_file,
    top_k=None,
    ignore_summaries=ignore_summaries,
    verbose=False,
)

Checking GT and Pred files matching...
Checking complete!
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.0989
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.2088
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.0839
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.2781
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.2781
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.2781


# END